# Analyse du sondage sur la précarité

Grandes questions sur l'architecture du rapport:

- est-ce qu'on fait une description puis analyse des facteurs influence pour chaque type de précarité ou bien plutot une descritpion générale avec nos indicateurs de précarité puis pleins de sous-parties explicatives?

- est-ce qu'on essaye de créer un indicatif/score de précarité? Ça peut être interessant et facile à utiliser pour les analyses

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import numpy as np
import string
import json
from statsmodels.stats import diagnostic
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp as mc
from sci_analysis import analyze
from scipy.stats import ttest_ind
import plotly.express as px


In [99]:
# Load the data

data = pd.read_csv("precarite.csv")
data.head()

,Horodateur,Score,De quel pays viens-tu (nationalité)?\n\nWhat is your country of origin (nationality)?,Quel est ton niveau d'études?\n\nWhat's your level of studies?,Quel est ton école?\n\nWhich school do you go to?,Quelle est ton expression de genre?\n\nWhat is your expression of genre?,Es-tu en échange à l'EPFL/ l'UNIL?\n\nAre you doing an exchange at EPFL/UNIL?,Combien as-tu d'enfants?\n\nHow many kids do you have?,"En moyenne, quelles sont tes dépenses mensuelles (en CHF) (en comptant logement, alimentation, loisir, santé, frais liés aux études)?\nApproximately, what are your monthly expenses (CHF) (taking into account housing, food, healthcare, hobbies and study fees)",Combien payes-tu ton loyer (CHF)? \n\nHow much is your rent? (CHF),...,Tes parents ont-ils fait des études supérieurs?\n\nDid your parents go to college/university?,Ta famille soutient-elle tes choix d'études / de carrière?\n\nDoes your family support you in your choice of studies/ of carrier?,Ta famille soutient-elle ton expression de genre/ ta sexualité?\n\nDoes your family support your expression of genre/ sexuality?,Ressens-tu une pression de réussite vis-à-vis de ta famille?\n\nDoes your family pressure you to succeed in your studies?,Vis-tu:\n\nDo you live:,"Si tu ne vis pas seul.e, quel type de relation entretiens-tu avec les personnes partageant ton logement?\n\nIf you don't live alone, how is the relationship with the people you live?",À quelle fréquence vois-tu des ami.es en dehors du contexte d'étude?\n\nHow often are you seeing your friends from school outside from lectures?,As-tu des ami.es avec qui tu peux travailler?\n\nDo you have friends you can work with?,"En cas de problème, as tu quelqu'un.e sur place sur qui tu peux compter? Pour discuter au téléphone, pour t'accompagner chez le docteur,...\nIn case of emergency, do you have someone close you can count on? Call to chat, go to the doctor with, ...",Est-ce que la sensation de solitude te pèse au quotidien?\n\nDoes the feeling of loneliness is part of your daily life?
0,10/04/2024 18:15:20,NaN,France,Master,EPFL,Woman,No,0.0,1700.0,900.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10/04/2024 18:18:58,NaN,France,Master,EPFL,Woman,No,0.0,1400.0,700.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10/04/2024 18:41:44,NaN,Benin,Master,EPFL,Woman,No,0.0,1700.0,807.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10/04/2024 18:52:43,NaN,Switzerland,Bachelor,EPFL,Man,No,0.0,NaN,600.0,...,Oui (un seul ou les deux) - Yes (one or both),Oui - Yes,Oui - Yes,Oui - Yes,En colocation - In flatsharing,3.0,Plusieurs fois par mois - Several times a month,Oui - Yes,Oui - Yes,Non - No
4,10/04/2024 18:59:28,NaN,France,Master,EPFL,Woman,Yes,0.0,1500.0,900.0,...,Oui (un seul ou les deux) - Yes (one or both),Oui - Yes,Oui - Yes,Oui - Yes,En colocation - In flatsharing,5.0,Plusieurs fois par semaine - Several times a week,Oui - Yes,Oui - Yes,Oui parfois - Yes sometimes


In [100]:
#Drop 2 premières colonnes
data.drop(columns=['Horodateur','Score'], inplace=True)


In [101]:
# Renommer les colonnes
data.columns=['Nationalité','Études','École','Genre','Échange','Enfants','Dépenses','Loyer','Revenus','Travail','Salaire','2bou','Bourses','Budget alimentaire','Manger bien et assez','Mealprep','Épiceries solidaires','Dépenser +','Ramen','Pk raviolis','OALC!','Assurance','Maladie','Médecin','Sport','Point santé','Pharmacie','Menstruations','Santé mentale','Avenir','Diagnostic','Dispositif aides','Famille','Études parents','Soutien familiale études','Soutien familiale genre','Pression réussite','Logement','Relation coloc','Freq pote','Work buddies','Amis ambulance','Solitude']
data.head()

,Nationalité,Études,École,Genre,Échange,Enfants,Dépenses,Loyer,Revenus,Travail,...,Études parents,Soutien familiale études,Soutien familiale genre,Pression réussite,Logement,Relation coloc,Freq pote,Work buddies,Amis ambulance,Solitude
0,France,Master,EPFL,Woman,No,0.0,1700.0,900.0,"Famille - Family, Bourse",0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,France,Master,EPFL,Woman,No,0.0,1400.0,700.0,"Bourses internationales (ERASMUS, CROUS) - Int...",0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Benin,Master,EPFL,Woman,No,0.0,1700.0,807.0,Famille - Family,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Switzerland,Bachelor,EPFL,Man,No,0.0,NaN,600.0,Famille - Family,2.0,...,Oui (un seul ou les deux) - Yes (one or both),Oui - Yes,Oui - Yes,Oui - Yes,En colocation - In flatsharing,3.0,Plusieurs fois par mois - Several times a month,Oui - Yes,Oui - Yes,Non - No
4,France,Master,EPFL,Woman,Yes,0.0,1500.0,900.0,"Salaire - Salary, Famille - Family",2.0,...,Oui (un seul ou les deux) - Yes (one or both),Oui - Yes,Oui - Yes,Oui - Yes,En colocation - In flatsharing,5.0,Plusieurs fois par semaine - Several times a week,Oui - Yes,Oui - Yes,Oui parfois - Yes sometimes


In [102]:
# Convertir les colonnes chiffre en numeric

data['Enfants']=pd.to_numeric(data['Enfants'])
data['Dépenses']=pd.to_numeric(data['Dépenses'])
data['Loyer']=pd.to_numeric(data['Loyer'])
data['Travail']=pd.to_numeric(data['Travail'])
data['Salaire']=pd.to_numeric(data['Salaire'])
data['Budget alimentaire']=pd.to_numeric(data['Budget alimentaire'])
data['Mealprep']=pd.to_numeric(data['Mealprep'])
data['Relation coloc']=pd.to_numeric(data['Relation coloc'])
data['Enfants']=pd.to_numeric(data['Enfants'])


In [103]:
#Colonnes à choix multiples: Revenus, santé mentale, pk ramen, sport

#Revenus

Famille=[]
Bourses_EPFL=[]
Bourses_canton=[]
Bourses_internat=[]
Travail=[]
Pret=[]

T=data['Revenus'].str.contains('Salaire')
for i in data.index:
    if T[i]==True:
        Travail.append(1)
    else:
        Travail.append(0)

data['Revenus_travail']=Travail


BE=data['Revenus'].str.contains('Bourses UNIL')
for i in data.index:
    if BE[i]==True:
        Bourses_EPFL.append(1)
    else:
        Bourses_EPFL.append(0)

data['Revenus_bourses_EPFL_UNIL']=Bourses_EPFL


BC=data['Revenus'].str.contains('Bourses cantonales')
for i in data.index:
    if BC[i]==True:
        Bourses_canton.append(1)
    else:
        Bourses_canton.append(0)

data['Revenus_bourses_canton']=Bourses_EPFL

BI=data['Revenus'].str.contains('Bourses internationales')
for i in data.index:
    if BI[i]==True:
        Bourses_internat.append(1)
    else:
        Bourses_internat.append(0)

data['Revenus_bourses_internationales']=Bourses_internat

F=data['Revenus'].str.contains('Famille')
for i in data.index:
    if F[i]==True:
        Famille.append(1)
    else:
        Famille.append(0)

data['Revenus_famille']=Famille


P=data['Revenus'].str.contains('Emprunt')
for i in data.index:
    if P[i]==True:
        Pret.append(1)
    else:
        Pret.append(0)

data['Revenus_emprunt']=Pret



In [105]:
#Santé mentale

Heureux=[]
Satisfait=[]
Neutre=[]
Triste=[]
Angry=[]
Stressed=[]
Anxieux=[]
Depressed=[]

H=data['Santé mentale'].str.contains('Heureux')
for i in data.index:
    if H[i]==True:
        Heureux.append(1)
    else:
        Heureux.append(0)

data['SM_heureux']=Heureux

S=data['Santé mentale'].str.contains('Satisfait')
for i in data.index:
    if S[i]==True:
        Satisfait.append(1)
    else:
        Satisfait.append(0)

data['SM_satisfait']=Satisfait

N=data['Santé mentale'].str.contains('Neutre')
for i in data.index:
    if N[i]==True:
        Neutre.append(1)
    else:
        Neutre.append(0)

data['SM_neutre']=Neutre

T=data['Santé mentale'].str.contains('Triste')
for i in data.index:
    if T[i]==True:
        Triste.append(1)
    else:
        Triste.append(0)

data['SM_triste']=Triste

St=data['Santé mentale'].str.contains('Stressé')
for i in data.index:
    if St[i]==True:
        Stressed.append(1)
    else:
        Stressed.append(0)

data['SM_stressé']=Stressed

Anx=data['Santé mentale'].str.contains('Anxieux')
for i in data.index:
    if Anx[i]==True:
        Anxieux.append(1)
    else:
        Anxieux.append(0)

data['SM_anxieux']=Anxieux

Dp=data['Santé mentale'].str.contains('Déprimé')
for i in data.index:
    if Dp[i]==True:
        Depressed.append(1)
    else:
        Depressed.append(0)

data['SM_déprimé']=Depressed


,Nationalité,Études,École,Genre,Échange,Enfants,Dépenses,Loyer,Revenus,Travail,...,Revenus_bourses_internationales,Revenus_famille,Revenus_emprunt,SM_heureux,SM_satisfait,SM_neutre,SM_triste,SM_stressé,SM_anxieux,SM_déprimé
0,France,Master,EPFL,Woman,No,0.0,1700.0,900.0,"Famille - Family, Bourse",0.0,...,0,1,0,1,0,0,0,1,0,0
1,France,Master,EPFL,Woman,No,0.0,1400.0,700.0,"Bourses internationales (ERASMUS, CROUS) - Int...",0.0,...,1,0,0,1,0,0,0,1,0,0
2,Benin,Master,EPFL,Woman,No,0.0,1700.0,807.0,Famille - Family,0.0,...,0,1,0,0,0,1,0,0,0,0
3,Switzerland,Bachelor,EPFL,Man,No,0.0,NaN,600.0,Famille - Family,2.0,...,0,1,0,1,1,0,0,0,1,0
4,France,Master,EPFL,Woman,Yes,0.0,1500.0,900.0,"Salaire - Salary, Famille - Family",2.0,...,0,1,0,0,1,0,0,0,0,0


In [106]:
#Pk raviolis

Envie=[]
Temps=[]
Argent=[]
Flemme=[]
Cuisiner=[]

Env=data['Pk raviolis'].str.contains('envie')
for i in data.index:
    if Env[i]==True:
        Envie.append(1)
    else:
        Envie.append(0)

data['PK_ravioli_envie']=Envie

Te=data['Pk raviolis'].str.contains('temps')
for i in data.index:
    if Te[i]==True:
        Temps.append(1)
    else:
        Temps.append(0)

data['PK_ravioli_temps']=Temps

Arg=data['Pk raviolis'].str.contains('argent')
for i in data.index:
    if Arg[i]==True:
        Argent.append(1)
    else:
        Argent.append(0)

data['PK_ravioli_argent']=Argent

Fle=data['Pk raviolis'].str.contains('flemme')
for i in data.index:
    if Fle[i]==True:
        Flemme.append(1)
    else:
        Flemme.append(0)

data['PK_ravioli_flemme']=Flemme

Cui=data['Pk raviolis'].str.contains('cuisiner')
for i in data.index:
    if Cui[i]==True:
        Cuisiner.append(1)
    else:
        Cuisiner.append(0)

data['PK_ravioli_cuisiner']=Cuisiner



In [107]:
#Sport

Gratuit=[]
Payant=[]
Pas_envie=[]
Pas_les_sous=[]
Pas_le_temps=[]
Pb_physique=[]

Grat=data['Sport'].str.contains('gratuitement')
for i in data.index:
    if Grat[i]==True:
        Gratuit.append(1)
    else:
        Gratuit.append(0)
data['Sport_gratuit']=Gratuit

Pay=data['Sport'].str.contains('budget')
for i in data.index:
    if Pay[i]==True:
        Payant.append(1)
    else:
        Payant.append(0)
data['Sport_payant']=Payant

PasE=data['Sport'].str.contains('envie')
for i in data.index:
    if PasE[i]==True:
        Pas_envie.append(1)
    else:
        Pas_envie.append(0)
data['Sport_pas_envie']=Pas_envie

PasS=data['Sport'].str.contains('financi')
for i in data.index:
    if PasS[i]==True:
        Pas_les_sous.append(1)
    else:
        Pas_les_sous.append(0)
data['Sport_pas_sous']=Pas_les_sous

PasT=data['Sport'].str.contains('temps')
for i in data.index:
    if PasT[i]==True:
        Pas_le_temps.append(1)
    else:
        Pas_le_temps.append(0)
data['Sport_pas_temps']=Pas_le_temps

PasP=data['Sport'].str.contains('physique')
for i in data.index:
    if PasP[i]==True:
        Pb_physique.append(1)
    else:
        Pb_physique.append(0)
data['Sport_pb_physique']=Pb_physique



## 1 - Faire l'état des lieux descriptifs

In [ ]:
# Histogrammes des dépenses mensuelles et des prix des loyers



In [ ]:
# Histogramme des sources de revenus


In [ ]:
# Heures par semaines travaillées et salaire horaire moyen

In [ ]:
# Fins de mois difficiles